In [3]:
import pandas as pd
import spacy
import json
from tqdm import tqdm

### Load data

In [38]:
lang = 'zh'
Set = 'train'
extractor = 'LM'

In [39]:
data= pd.read_csv(f'../Dataset/{Set}.csv')
data.lang.unique()

array(['bn', 'de', 'en', 'es', 'fa', 'fr', 'hi', 'it', 'pt', 'sv', 'uk',
       'zh'], dtype=object)

In [40]:
data = data[data['lang']==lang]


### Information Extraction Pipeline

In [41]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-base-ner-hrl")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)


In [42]:
#import the pipeline class


from InformationExtraction import InformationExtractionPipeline


# example spacy extractor function
NER = spacy.load("en_core_web_lg")
def tag_extraction_from_spacy(sen, model = NER):
    
    annotated = model(sen)
    extracted_names = [word.text for word in annotated.ents 
                       if word.label_=='PERSON' or word.label_=='ORG'or word.label_=='GPE']
    
    
    return extracted_names


def tag_extraction_from_LM(sen, lang, model = nlp):
    
    if lang=='zh':
        sen = sen.replace(" ","")
    
    ner_results = model(sen)
    extracted_names = []

    for idx in range(len(ner_results)):
        if ner_results[idx]['entity'][0] == 'B':
            start = ner_results[idx]['start']
            end = ner_results[idx]['end']
            j = idx+1
            while j < len(ner_results):
                if ner_results[j]['entity'][0] == 'B':
                    break
                elif ner_results[j]['entity'][0] == 'I':
                    end = ner_results[j]['end']
                j+=1
            idx = j
        
            extracted_names.append(sen[start:end].strip())
    
    
    return extracted_names


# example extractor function that uses training labels 
sent_to_tag = dict(zip(data['sent'],data['labels']))
def tag_extraction_from_tags(sent, lang, sent_to_tag=sent_to_tag):

    tags = sent_to_tag[sent]
        
    sentsWithtags = [(s,t) for s,t in zip(sent.split(),tags.split())]
        
        
    entity_list = []
    for i,item in enumerate(sentsWithtags):
        if 'B-' in item[1]:
            j = i
            entity = []
            while j<len(sentsWithtags):
                if sentsWithtags[j][1] =='O':
                    break
                entity.append(sentsWithtags[j][0])
                j+=1
            i = j
             
            if lang=='zh':
                entity_list.append("".join(entity))
            
            else:
                entity_list.append(" ".join(entity))
            
    

    return entity_list




In [43]:
#create pipline object:
#param: extractor: an entity extractor function that returns all the entities from a sentence
#param: max_sen: define the number of sentences to be added for each detected entity
#param: lang: define language. needed for wikipedia api
#param: saveJson: whether to save extracted informaton as json file. Saves time if needed to run the pipeline again
#param: loadJson: if you have saved a json file and want to use it
#param: jsonPath: define saved json file path


infoPipeline = InformationExtractionPipeline(extractor = tag_extraction_from_tags if extractor=='tags' else tag_extraction_from_LM, 
                                        max_sen = 2, lang = lang, 
                                        loadJson = True, jsonPath=f'wiki-info-{lang}-{Set}.json',
                                        saveJson=False, saveJsonpath=f'wiki-info-{lang}-{Set}.json')

In [45]:
#call pipline and provide list of sentences as argument

augmented = infoPipeline(data['sent'].values.tolist())

  7%|██▊                                     | 685/9758 [02:17<32:42,  4.62it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?action=query&prop=info&titles=magna&inprop=protection%7Ctalkid%7Cwatched%7Cwatchers%7Cvisitingwatchers%7Cnotificationtimestamp%7Csubjectid%7Curl%7Creadable%7Cpreload%7Cdisplaytitle&format=json&redirects=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1dff3e0af0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))


 21%|████████▎                              | 2090/9758 [09:47<31:07,  4.11it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 26%|█████████▉                             | 2493/9758 [13:25<24:05,  5.03it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


 80%|███████████████████████████████        | 7766/9758 [29:40<06:29,  5.11it/s]

HTTPSConnectionPool(host='zh.wikipedia.org', port=443): Read timed out. (read timeout=10.0)


100%|███████████████████████████████████████| 9758/9758 [35:51<00:00,  4.54it/s]


### Info Percentage

In [46]:
data['augmented_sen'] = augmented
temp = data[data['sent']!=data['augmented_sen']]


In [47]:
info_percent = temp.shape[0]/data.shape[0]
print(f"Info Percentage: {info_percent*100:.2f}%")

Info Percentage: 30.14%


In [52]:
import re

def separate_funct(sent):
    

    regex = re.compile("[\u4e00-\u9fff]|\:|\.\d+|\.|·\S+·?|·|·「|」·|「|」|[^\u4e00-\u9fff\s]+")

#     return regex.findall(sent)
    return " ".join(regex.findall(sent)).strip()

In [75]:
# if lang=='zh':
data['augmented_sen'] = data['augmented_sen'].apply(lambda x: separate_funct(x))
data['sent'] = data['sent'].apply(lambda x: separate_funct(x))

In [76]:
# data['augmented_sen'] = augmented
temp = data[data['sent']!=data['augmented_sen']]
info_percent = temp.shape[0]/data.shape[0]
print(f"Info Percentage: {info_percent*100:.2f}%")

Info Percentage: 30.16%


In [68]:
count = 0
# Set = 'train'
# lang = 'zh'

# data = pd.read_csv(f'../Dataset/{Set}-wiki-{lang}-LM.csv')
# data = data[data['lang']==lang]
for s,l in zip(data['sent'],data['labels']):
    
#     print(s)
    if len(separate_funct(s).split())!=len(l.split()):
        print(s)
        count+=1
count

0

In [60]:
data['augmented_sen'].iloc[35]

'州 长 山 本 一 太 说 他 希 望 此 举 能 传 达 积 极 的 信 息 并 促 进 多 样 性 . 山 本 一 太 （1958 年 1 月 24 日 －）， 日 本 政 治 家 ， 群 馬 縣 知 事 ， 前 自 由 民 主 黨 黨 員 。 出 身 於 群 馬 縣 高 崎 市 （ 舊 群 馬 郡 倉 淵 村 ）。1995 年 至 今 連 續 當 選 4 屆 參 議 院 議 員 。 中 央 大 學 大 學 院 客 席 教 授 。 歷 任 外 務 政 務 次 官 、 參 議 院 外 交 防 衛 委 員 長 、 外 務 副 大 臣 等 職 務 。2012 年 首 次 入 閣 ， 被 任 命 為 第 二 次 安 倍 內 閣 的 内 閣 府 特 命 擔 當 大 臣 （ 科 學 技 術 政 策 ·北方及北方對策·宇宙政策擔當）父親山本富雄也是政治家，曾擔任農林水產大臣。'

### Save Augmented Data

In [61]:
data.to_csv(f'../Dataset/{Set}-wiki-{lang}-{extractor}.csv',index=False)

In [62]:
# get_info('dev','de')
Set = 'train'
lang = 'zh'

data = pd.read_csv(f'../Dataset/{Set}-wiki-{lang}-LM.csv')
data = data[data['lang']==lang]

# data['sent'] = data['sent'].apply(lambda x: separate_funct(x))
# data.to_csv(f'../Dataset/{Set}-wiki-{lang}-LM.csv',index=False)
# sent_to_tag = dict(zip(data['sent'],data['labels']))

In [73]:
data['augmented_sen'].tolist()[1]

'新 國 家 黨 ( saenuri dang ) 由 朴 槿 惠 领 导 . 朴 槿 惠 （ 韩 语 ：박근혜／ 朴 槿 惠 park geun-hye；1952 年 2 月 2 日 －）， 大 韩 民 国 第 18 任 總 統 ， 前 总 统 朴 正 熙 长 女 ， 韩 国 宪 政 史 上 首 位 女 性 总 统 ， 首 位 在 韩 战 爆 发 后 出 生 的 总 统 和 首 位 “ 二 代 总 统 ”， 韩 国 目 前 唯 一 曾 代 理 过 第 一 夫 人 的 总 统 。 朴 槿 惠 也 是 韩 国 自 1987 年 開 始 民 主 化 以 來 唯 一 一 位 在 總 統 選 舉 中 得 票 率 過 半 的 總 統 。 朴 槿 惠 1952 年 2 月 2 日 出 生 于 朝 鲜 战 争 时 期 的 韓 國 庆 尚 北 道 大 邱 市 ，1974 年 从 西 江 大 学 电 子 工 程 系 毕 业 后 ， 留 学 法 国 。 期 间 ， 其 母 陆 英 修 在 1974 年 8 月 15 日 韩 国 光 复 29 周 年 纪 念 典 礼 上 被 朝 鲜 间 谍 文 世 光 杀 害 。 由 于 朴 正 熙 无 意 再 娶 ， 朴 槿 惠 代 替 其 母 履 行 韓 國 第 一 夫 人 的 职 责 。1979 年 10 月 26 日 ， 朴 正 熙 被 韓 國 中 央 情 报 部 部 长 金 载 圭 枪 杀 后 ， 她 带 着 妹 妹 和 弟 弟 从 青 瓦 台 搬 回 位 于 新 堂 洞 的 老 宅 。1997 年 亚 洲 金 融 危 机 时 期 ， 她 结 束 隐 居 生 活 正 式 进 入 政 坛 。 此 后 她 先 后 出 任 大 国 家 党 副 代 表 以 及 代 表 ， 连 续 5 次 当 选 国 会 议 员 ，2012 年 ， 以 韩 国 总 统 选 举 历 史 上 首 次 超 过 半 数 的 得 票 率 （51.6%） 成 功 当 选 韩 国 第 18 任 总 统 ， 成 为 韩 国 乃 至 东 北 亚 地 區 历 史 上 首 位 民 選 女 性 国 家 元 首 。2016 年 12 月 9 日 ， 朴 槿 惠 因 崔 順 實 事 件 而 被 大 韓 民 國 國 會 通 過 弹 劾 ，2017 年 3 月 10 日 ， 韓 國 宪 法 法 院 裁

In [74]:
sent = data['sent'].tolist()[1]
sent

'新  國  家  黨  (  saenuri  dang  )  由  朴  槿  惠  领  导  . '

In [17]:
sent = data['sent'].tolist()[127]

In [18]:
sent

'在  亨  利  ·  德  保  蒙  的  指  挥  下  灰  色  在  围  困  中  进  行  了  战  斗  . '

In [19]:
out = tag_extraction_from_LM(sent,lang)
out

['亨利·德保蒙']

In [20]:
out = tag_extraction_from_tags(sent,lang)
out

['亨利·德保蒙']

In [29]:
labels = data['labels'].tolist()[4]
labels

' O  O  O  B-Politician  I-Politician  I-Politician  I-Politician  I-Politician  I-Politician  I-Politician  O  B-OtherPER  I-OtherPER  I-OtherPER  I-OtherPER  I-OtherPER  I-OtherPER  I-OtherPER  O  O  O  O  O  O  O  O  O  O  O  O  O  O'

In [39]:
sent = data['augmented_sen'].tolist()[1]
sent
# separate_funct(sent)

'新 國 家 黨 ( saenuri dang ) 由 朴 槿 惠 领 导 . 自 由 韓 國 黨 （ 韩 语 ：자유한국당／ 自 由 韓 國 黨 jayuhangukdang）， 简 称 自 韓 黨 或 韓 國 黨 ， 大 韓 民 國 已 解 散 的 保 守 主 义 右 翼 政 黨 ， 原 名 大 国 家 党 （ 韩 语 ：한나라당／한나라 黨 hannaradang），2012 年 更 名 為 新 世 界 党 （ 韩 语 ：새누리당／새누리 黨 saenuridang），2017 年 更 名 为 最 終 黨 名 。2020 年 2 月 17 日 ， 與 新 保 守 黨 、 向 未 來 前 進 4.0 合 併 為 未 來 統 合 黨 而 解 散 。 支 持 者 主 要 为 40 岁 以 上 中 年 和 老 年 人 ， 以 及 慶 尚 道 、 釜 山 和 大 邱 一 帶 。 歷 經 2016 年 崔 順 實 干 政 醜 聞 及 文 在 寅 政 府 執 政 後 ，2018 年 地 方 選 舉 慘 敗 ， 失 去 多 個 廣 域 市 和 地 方 議 會 的 席 位 ， 但 解 散 前 仍 為 韓 國 政 壇 的 第 一 大 在 野 黨 。 朴 槿 惠 （ 韩 语 ：박근혜／ 朴 槿 惠 park geun-hye；1952 年 2 月 2 日 －）， 大 韩 民 国 第 18 任 總 統 ， 前 总 统 朴 正 熙 长 女 ， 韩 国 宪 政 史 上 首 位 女 性 总 统 ， 首 位 在 韩 战 爆 发 后 出 生 的 总 统 和 首 位 “ 二 代 总 统 ”， 韩 国 目 前 唯 一 曾 代 理 过 第 一 夫 人 的 总 统 。 朴 槿 惠 也 是 韩 国 自 1987 年 開 始 民 主 化 以 來 唯 一 一 位 在 總 統 選 舉 中 得 票 率 過 半 的 總 統 。 朴 槿 惠 1952 年 2 月 2 日 出 生 于 朝 鲜 战 争 时 期 的 韓 國 庆 尚 北 道 大 邱 市 ，1974 年 从 西 江 大 学 电 子 工 程 系 毕 业 后 ， 留 学 法 国 。 期 间 ， 其 母 陆 英 修 在 1974 年 8 月 15 日 韩 国 光 复 29 周 年 纪 念 典 礼 上 被 朝 鲜 间 谍 文 世 光 杀 害 。 由 于 朴 正 熙 

In [84]:
# sent = "她  在  1947  年  通  过  清  唱  套  曲  设  置  阿  尔  图  尔  ·  兰  波  诗  歌  saisons  châteaux  的  诗  ."
count = 0
Set = 'train'
lang = 'zh'

data = pd.read_csv(f'../Dataset/{Set}-wiki-{lang}-LM.csv')
data = data[data['lang']==lang]
for s,l in zip(data['sent'],data['labels']):
    
#     print(s)
    if len(separate_funct(s).split())!=len(l.split()):
#         print(s)
        count+=1
count

2081

In [31]:
separate_funct('新  國  家  黨  (  saenuri  dang  )  由  朴  槿  惠  领  导  . 自由韓國黨（韩语：자유한국당／自由韓國黨 jayuhangukdang），简称自韓黨或韓國黨，大韓民國已解散的保守主义右翼政黨，原名大国家党（韩语：한나라당／한나라黨 hannaradang），2012年更名為新世界党（韩语：새누리당／새누리黨 saenuridang），2017年更名为最終黨名。2020年2月17日，與新保守黨、向未來前進4.0合併為未來統合黨而解散。支持者主要为40岁以上中年和老年人，以及慶尚道、釜山和大邱一帶。歷經2016年崔順實干政醜聞及文在寅政府執政後，2018年地方選舉慘敗，失去多個廣域市和地方議會的席位，但解散前仍為韓國政壇的第一大在野黨。 朴槿惠（韩语：박근혜／朴槿惠 park geun-hye；1952年2月2日－），大韩民国第18任總統，前总统朴正熙长女，韩国宪政史上首位女性总统，首位在韩战爆发后出生的总统和首位“二代总统”，韩国目前唯一曾代理过第一夫人的总统。朴槿惠也是韩国自1987年開始民主化以來唯一一位在總統選舉中得票率過半的總統。朴槿惠1952年2月2日出生于朝鲜战争时期的韓國庆尚北道大邱市，1974年从西江大学电子工程系毕业后，留学法国。期间，其母陆英修在1974年8月15日韩国光复29周年纪念典礼上被朝鲜间谍文世光杀害。由于朴正熙无意再娶，朴槿惠代替其母履行韓國第一夫人的职责。1979年10月26日，朴正熙被韓國中央情报部部长金载圭枪杀后，她带着妹妹和弟弟从青瓦台搬回位于新堂洞的老宅。1997年亚洲金融危机时期，她结束隐居生活正式进入政坛。此后她先后出任大国家党副代表以及代表，连续5次当选国会议员，2012年，以韩国总统选举历史上首次超过半数的得票率（51.6%）成功当选韩国第18任总统，成为韩国乃至东北亚地區历史上首位民選女性国家元首。2016年12月9日，朴槿惠因崔順實事件而被大韓民國國會通過弹劾，2017年3月10日，韓國宪法法院裁定弹劾成立，使她成为大韓民國歷史上首位因彈劾而下台的總統')

'新 國 家 黨 ( saenuri dang ) 由 朴 槿 惠 领 导 . 自 由 韓 國 黨 （ 韩 语 ：자유한국당／ 自 由 韓 國 黨 jayuhangukdang）， 简 称 自 韓 黨 或 韓 國 黨 ， 大 韓 民 國 已 解 散 的 保 守 主 义 右 翼 政 黨 ， 原 名 大 国 家 党 （ 韩 语 ：한나라당／한나라 黨 hannaradang），2012 年 更 名 為 新 世 界 党 （ 韩 语 ：새누리당／새누리 黨 saenuridang），2017 年 更 名 为 最 終 黨 名 。2020 年 2 月 17 日 ， 與 新 保 守 黨 、 向 未 來 前 進 4.0 合 併 為 未 來 統 合 黨 而 解 散 。 支 持 者 主 要 为 40 岁 以 上 中 年 和 老 年 人 ， 以 及 慶 尚 道 、 釜 山 和 大 邱 一 帶 。 歷 經 2016 年 崔 順 實 干 政 醜 聞 及 文 在 寅 政 府 執 政 後 ，2018 年 地 方 選 舉 慘 敗 ， 失 去 多 個 廣 域 市 和 地 方 議 會 的 席 位 ， 但 解 散 前 仍 為 韓 國 政 壇 的 第 一 大 在 野 黨 。 朴 槿 惠 （ 韩 语 ：박근혜／ 朴 槿 惠 park geun-hye；1952 年 2 月 2 日 －）， 大 韩 民 国 第 18 任 總 統 ， 前 总 统 朴 正 熙 长 女 ， 韩 国 宪 政 史 上 首 位 女 性 总 统 ， 首 位 在 韩 战 爆 发 后 出 生 的 总 统 和 首 位 “ 二 代 总 统 ”， 韩 国 目 前 唯 一 曾 代 理 过 第 一 夫 人 的 总 统 。 朴 槿 惠 也 是 韩 国 自 1987 年 開 始 民 主 化 以 來 唯 一 一 位 在 總 統 選 舉 中 得 票 率 過 半 的 總 統 。 朴 槿 惠 1952 年 2 月 2 日 出 生 于 朝 鲜 战 争 时 期 的 韓 國 庆 尚 北 道 大 邱 市 ，1974 年 从 西 江 大 学 电 子 工 程 系 毕 业 后 ， 留 学 法 国 。 期 间 ， 其 母 陆 英 修 在 1974 年 8 月 15 日 韩 国 光 复 29 周 年 纪 念 典 礼 上 被 朝 鲜 间 谍 文 世 光 杀 害 。 由 于 朴 正 熙 

161054    內  穆  爾  ·  哈  克  政  治  顾  问  (  生  于  1949  年...
161055    新  國  家  黨  (  saenuri  dang  )  由  朴  槿  惠  领...
161056    拿  破  仑  还  委  托  a  庇  护  七  世  肖  像  大  卫  特...
161057    约  翰  ·  兰  登  (  民  主  共  和  党  )  16  325  (...
161058    沃  克  以  佛  萊  德  ·  漢  普  頓  和  弗  莱  德  ·  希...
                                ...                        
170807    肝  性  腦  病  變  中  也  指  示  乳  lactulose  . 肝性腦...
170808    他  撰  写  了  许  多  关  于  同  性  恋  的  文  章  和  书...
170809    为  这  首  歌  支  付  的  钱  捐  赠  给  了  艾  滋  病  研...
170810    白  细  胞  是  發  炎  响  应  中  非  常  重  要  的  药  物...
170811    更  强  烈  的  感  染  可  能  导  致  严  重  的  發  炎  周...
Name: augmented_sen, Length: 9758, dtype: object